In [1]:
import sentencepiece as spm
#READING FILE
with open('hi_100.txt', 'r', encoding='utf-8') as file:
    contents = file.read()
# print(contents)

In [2]:
hindi_vowels = ['अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 'अं', 'अः', 'अँ']
matra = ['ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ो', 'ै', 'ौ', 'ृ', 'ॄ', 'ॉ', 'ं','्','ँ']
hindi_consonants = [
    'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श',
    'ष', 'स', 'ह', 'ळ', 'क्ष', 'ज्ञ'
]
mapping = {'ा': 'आ', 'ि': 'इ', 'ी': 'ई', 'ु': 'उ', 'ू': 'ऊ', 'ृ': 'ऋ', 'ॄ': 'ॠ', 'ॅ': 'ऍ', 'े': 'ए', 'ै': 'ऐ', 'ो': 'ओ', 'ौ': 'औ', 'ं': 'अं', 'ः': 'अः', 'ँ': 'अँ'}

## FUNCTION FOR FINDING CHARACTERS:

In [3]:
def Getcharacters(tokens):
    characters = []
    for word in tokens:
        s = []
        for ch in word:
            if ch=="▁" :  continue
            if ch == '्' and len(word)==0: continue
            elif ch in hindi_vowels: s.append(ch)
            elif ch in hindi_consonants:
                s.append(ch + '्')
                s.append('अ')
            elif ch in matra:
                if len(s) and s[-1]=='अ': s.pop()
                if mapping.get(ch) is not None:
                    s.append(mapping.get(ch))
        if len(s):
              characters.append(s)
    return characters

## FUNCTION FOR FINDING SYLLABLES:

In [4]:
#Get syllables
def Getsyllables(tokens):
    s = ""
    syllables = []
    for w in tokens:
        syllable=[]
        s = ""
        for ch in w:
            if ch=="▁" :  continue
            if ch == '्' and len(s)==0: continue
            if ch in hindi_consonants:
                if(len(s) and s[-1]!='्'):
                    syllable.append(s)
                    s = ""
                s = s + ch
            elif ch in hindi_vowels:
                s = s + ch
                syllable.append(s)
                s = ""
            elif (ch in matra) :
                if(len(s) and s[-1] in matra):
                    syllable.append(s)
                    syllable.append(mapping.get(ch))
                    s = ""
                else: s = s + ch
        if len(s):
            syllable.append(s) 
        if len(syllable):
            syllables.append(syllable)    
    for i in range(0, len(syllables)):
        for j in range(0, len(syllables[i])):
            if syllables[i][j] in matra:
                syllables[i][j] = mapping.get(syllables[i][j])
    return syllables

In [5]:
def clean_token(tokens1):
    tokens = []
    for word in tokens1:
        s=""
        for c in word:
            if (c not in matra) and (c not in hindi_consonants) and (c not in hindi_vowels): continue
            else: s = s + c
        if(len(s)): tokens.append(s)
    return tokens

## RANK BY FREQUENCY:

In [6]:
#rank by frequency
def count_ch(ch):
    freq = {}
    for word in ch:
        for c in word:
            if c in freq:
                freq[c] += 1
            else:
                freq[c] = 1
    sorted_dict = sorted(freq.items(), key=lambda item: item[1], reverse = True)
    return sorted_dict[:20]

## RANK BY FREQUENCY FOR TOKENS:

In [7]:
#rank by frequency for tokens
def count_t(tokens):
    token_freq = {}
    for word in tokens:
        if word in token_freq:
            token_freq[word] += 1
        else:
            token_freq[word] = 1
    sorted_dict = sorted(token_freq.items(), key=lambda item: item[1], reverse = True)
    return sorted_dict[:20]

## BIGRAM:

In [8]:
#bigram
def create_bi(lst):
    bigram_ch = {}
    for word in lst:
        for i in range (0, len(word)-1):
            if word[i] is None or word[i+1] is None: continue
            elif (word[i]+word[i+1]) in bigram_ch:
                bigram_ch[word[i]+word[i+1]] += 1
            else:
                bigram_ch[word[i]+word[i+1]] = 1
    bigram_ch = sorted(bigram_ch.items(), key=lambda item: item[1], reverse = True)
    return bigram_ch[:20]

## BIGRAM FOR TOKENS:

In [9]:
def create_bi_tokens(lst):
    bigram_token = {}
    for i in range (0, len(lst)-1):
        if lst[i] is None or lst[i+1] is None: continue
        elif (lst[i]+lst[i+1]) in bigram_token:
            bigram_token[lst[i]+lst[i+1]] += 1
        else:
            bigram_token[lst[i]+lst[i+1]] = 1
    bigram_token = sorted(bigram_token.items(), key=lambda item: item[1], reverse = True)
    return bigram_token[:20]

## FUNCTION FOR FINDING ALL FREQUENCIES:

In [10]:
def processing(tokens1):
    print("GENETRATED TOKENS (TOP 50): ", tokens1[:50])
    print("\n")
    ch = Getcharacters(tokens1)
#     print("UNICODE CORRECTION: ", ch)
#     print("\n")
    sy = Getsyllables(tokens1)
#     print("SYLLABLES: ", sy)
#     print("\n")
    T = clean_token(tokens1)
#     print("CLEANED TOKENS: ", T)
#     print("\n")
    count1 = count_ch(ch)
    print("TOP 20 UNIGRAM CHARACTERS: ", count1)
    print("\n")
    count2 = count_ch(sy)
    print("TOP 20 UNIGRAM SYLLABLES: ", count2)
    print("\n")
    count3 = count_t(T)
    print("TOP 20 UNIGRAM TOKENS:", count3)
    print("\n")
    bi_ch = create_bi(ch)
    print("TOP 20 BIGRAM CHARACTERS:", bi_ch)
    print("\n")
    bi_syl = create_bi(sy)
    print("TOP 20 BIGRAM SYLLABLES: ", bi_syl)
    print("\n")
    bi_token = create_bi_tokens(T)
    print("TOP 20 BIGRAM TOKENS: ", bi_token)

## UNIGRAM TOKENIZER FOR VOCAB_SIZE - 1000

In [11]:
import sentencepiece as spm
corpus_file = "hi_100.txt"
model_prefix = "hindi_unigram"
vocab_size = 1000 
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, vocab_size=vocab_size, model_type='unigram')

In [12]:
sp = spm.SentencePieceProcessor()
sp.load(f"{model_prefix}.model")
tokens1 = sp.encode_as_pieces(contents)

In [13]:
print("UNIGRAM TOKENIZER: ")
processing(tokens1)

UNIGRAM TOKENIZER: 
GENETRATED TOKENS (TOP 50):  ['▁आ', 'वे', 'द', 'न', '▁करने', '▁की', '▁आखिर', 'ी', '▁ता', 'री', 'ख', '▁3', '1', '▁जनवरी', ',', '▁20', '20', '▁है', '।', '▁इतन', 'ी', '▁दु', 'आ', '▁कर', '▁दो', '▁हमारे', '▁लिए', '▁कि', '▁ज', 'ित', 'ना', '▁प', '्या', 'र', '▁दुनिया', '▁ने', '▁आपको', '▁दिया', '▁है', ',', '▁ब', 'स', '▁उ', 'त', 'ना', '▁ही', '▁हम', 'ें', '▁भी', '▁मिल']


TOP 20 UNIGRAM CHARACTERS:  [('अ', 9395580), ('आ', 2991109), ('ए', 2318442), ('क्', 2221101), ('र्', 2115631), ('ई', 1460305), ('इ', 1432973), ('न्', 1334418), ('स्', 1283708), ('अं', 1201207), ('ह्', 1133159), ('म्', 1053237), ('त्', 980066), ('ल्', 919917), ('ओ', 896588), ('प्', 805896), ('य्', 753228), ('व्', 624743), ('द्', 607633), ('उ', 587149)]


TOP 20 UNIGRAM SYLLABLES:  [('र', 1295862), ('अं', 1092392), ('क', 682323), ('न', 636034), ('स', 597900), ('इ', 564539), ('आ', 495701), ('प', 466246), ('ए', 458259), ('ल', 455106), ('म', 432836), ('के', 400441), ('त', 397464), ('ह', 351259), ('ने', 323222), ('

## UNIGRAM TOKENIZER FOR VOCAB_SIZE - 2000 

In [14]:
import sentencepiece as spm
corpus_file = "hi_100.txt"
model_prefix = "hindi_unigram2"
vocab_size = 2000 
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, vocab_size=vocab_size, model_type='unigram')

In [15]:
model_prefix = "hindi_unigram2"
sp = spm.SentencePieceProcessor()
sp.load(f"{model_prefix}.model")
tokens1 = sp.encode_as_pieces(contents)

In [16]:
print("UNIGRAM TOKENIZER FOR VOCAB SIZE 2000: ")
processing(tokens1)

UNIGRAM TOKENIZER FOR VOCAB SIZE 2000: 
GENETRATED TOKENS (TOP 50):  ['▁आवेदन', '▁करने', '▁की', '▁आखिर', 'ी', '▁तारीख', '▁3', '1', '▁जनवरी', ',', '▁20', '20', '▁है', '।', '▁इतन', 'ी', '▁द', 'ुआ', '▁कर', '▁दो', '▁हमारे', '▁लिए', '▁कि', '▁ज', 'ित', 'ना', '▁प्यार', '▁दुनिया', '▁ने', '▁आपको', '▁दिया', '▁है', ',', '▁बस', '▁उ', 'त', 'ना', '▁ही', '▁हमें', '▁भी', '▁मिल', '▁जाए', '|', '”', '▁मोदी', '▁सरकार', '▁के', '▁पहले', '▁कार्य', 'का']


TOP 20 UNIGRAM CHARACTERS:  [('अ', 8555645), ('आ', 2991109), ('ए', 2318442), ('क्', 2221101), ('र्', 2115631), ('ई', 1460305), ('इ', 1432973), ('न्', 1334418), ('स्', 1283708), ('अं', 1201207), ('ह्', 1133159), ('म्', 1053237), ('त्', 980066), ('ल्', 919917), ('ओ', 896588), ('प्', 805896), ('य्', 753228), ('व्', 624743), ('द्', 607633), ('उ', 587149)]


TOP 20 UNIGRAM SYLLABLES:  [('र', 1164867), ('अं', 1064967), ('क', 663284), ('न', 593532), ('स', 566728), ('प', 432451), ('इ', 416704), ('ल', 413140), ('के', 399027), ('आ', 374444), ('त', 371638), ('म', 3616

## BPE TOKENIZER FOR VOCAB_SIZE -  1000 

In [17]:
corpus_file = "hi_100.txt"
model_prefix = "hindi_bpe"
vocab_size = 1000 
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, vocab_size=vocab_size, model_type='bpe')

In [18]:
model_prefix = "hindi_bpe"
sp_b1 = spm.SentencePieceProcessor()
sp_b1.load(f"{model_prefix}.model")
tokens2 = sp_b1.encode_as_pieces(contents)
processing(tokens2)

GENETRATED TOKENS (TOP 50):  ['▁आ', 'वे', 'दन', '▁करने', '▁की', '▁आ', 'ख', 'ि', 'री', '▁त', 'ारी', 'ख', '▁3', '1', '▁जन', 'व', 'री', ',', '▁20', '2', '0', '▁है', '।', '▁इ', 'त', 'नी', '▁दु', 'आ', '▁कर', '▁दो', '▁हम', 'ारे', '▁लिए', '▁कि', '▁ज', 'ित', 'ना', '▁प', '्य', 'ार', '▁दु', 'निया', '▁ने', '▁आपको', '▁दिया', '▁है', ',', '▁ब', 'स', '▁उ']


TOP 20 UNIGRAM CHARACTERS:  [('अ', 9494781), ('आ', 2991109), ('ए', 2318442), ('क्', 2221101), ('र्', 2115631), ('ई', 1460305), ('इ', 1432973), ('न्', 1334418), ('स्', 1283708), ('अं', 1201207), ('ह्', 1133159), ('म्', 1053237), ('त्', 980066), ('ल्', 919917), ('ओ', 896588), ('प्', 805896), ('य्', 753228), ('व्', 624743), ('द्', 607633), ('उ', 587149)]


TOP 20 UNIGRAM SYLLABLES:  [('र', 1252342), ('अं', 1104660), ('क', 709936), ('आ', 675829), ('इ', 661042), ('न', 607095), ('स', 580344), ('प', 506804), ('म', 479719), ('ल', 461669), ('ए', 426746), ('त', 421137), ('के', 393586), ('ह', 358310), ('ने', 325957), ('ब', 325582), ('ग', 302023), ('ज', 2974

## BPE TOKENIZER FOR VOCAB_SIZE - 2000 

In [19]:
import sentencepiece as spm
corpus_file = "hi_100.txt"
model_prefix = "hindi_bpe2"
vocab_size = 2000 
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix=model_prefix, vocab_size=vocab_size, model_type='bpe')

In [20]:
model_prefix = "hindi_bpe2"
sp_b3 = spm.SentencePieceProcessor()
sp_b3.load(f"{model_prefix}.model")
tokens3 = sp_b3.encode_as_pieces(contents)
print("BPE TOKENIZER FOR VOCAB SIZE 2000: ")
processing(tokens3)

BPE TOKENIZER FOR VOCAB SIZE 2000: 
GENETRATED TOKENS (TOP 50):  ['▁आ', 'वेदन', '▁करने', '▁की', '▁आख', 'ि', 'री', '▁त', 'ारी', 'ख', '▁3', '1', '▁जनवरी', ',', '▁20', '20', '▁है', '।', '▁इत', 'नी', '▁दु', 'आ', '▁कर', '▁दो', '▁हमारे', '▁लिए', '▁कि', '▁ज', 'ित', 'ना', '▁प', '्य', 'ार', '▁दुनिया', '▁ने', '▁आपको', '▁दिया', '▁है', ',', '▁बस', '▁उ', 'त', 'ना', '▁ही', '▁हमें', '▁भी', '▁मिल', '▁जाए', '|', '”']


TOP 20 UNIGRAM CHARACTERS:  [('अ', 8643728), ('आ', 2991109), ('ए', 2318442), ('क्', 2221101), ('र्', 2115631), ('ई', 1460305), ('इ', 1432973), ('न्', 1334418), ('स्', 1283708), ('अं', 1201207), ('ह्', 1133159), ('म्', 1053237), ('त्', 980066), ('ल्', 919917), ('ओ', 896588), ('प्', 805896), ('य्', 753228), ('व्', 624743), ('द्', 607633), ('उ', 587149)]


TOP 20 UNIGRAM SYLLABLES:  [('र', 1164091), ('अं', 1075390), ('क', 656765), ('न', 576728), ('स', 563096), ('इ', 502138), ('आ', 489918), ('प', 450175), ('ल', 421939), ('के', 402872), ('म', 390011), ('त', 370312), ('ह', 334884), ('ने', 3259

## mBERT TOKENIZER FOR MAX_LENGTH - 1000 

In [ ]:
# cleaned = ""
# for word in contents:
#     s=""
#     for c in word:
#         if c == ' ': s = s+c
#         elif (c not in matra) and (c not in hindi_consonants) and (c not in hindi_vowels): continue
#         else: s = s + c
#     if(len(s)): cleaned += s
# print(cleaned)

In [23]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokens5 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(contents, max_length=1000, truncation=True, padding='max_length')))

In [24]:
print("mBERT TOKENIZER FOR VOCAB SIZE 1000: ")
processing(tokens5)

mBERT TOKENIZER FOR VOCAB SIZE 1000: 
GENETRATED TOKENS (TOP 50):  ['[CLS]', 'आ', '##वे', '##दन', 'करने', 'की', 'आ', '##ख', '##िर', '##ी', 'त', '##ारी', '##ख', '31', 'जनवरी', ',', '2020', 'है', '।', 'इ', '##तन', '##ी', 'दु', '##आ', 'कर', 'दो', 'हम', '##ारे', 'लिए', 'कि', 'ज', '##ित', '##ना', 'प', '##्या', '##र', 'दुनिया', 'ने', 'आ', '##प', '##को', 'दिया', 'है', ',', 'ब', '##स', 'उ', '##तन', '##ा', 'ही']


TOP 20 UNIGRAM CHARACTERS:  [('अ', 658), ('आ', 171), ('क्', 158), ('ए', 147), ('र्', 121), ('इ', 105), ('न्', 84), ('अं', 76), ('स्', 75), ('ई', 72), ('ह्', 71), ('ल्', 67), ('त्', 57), ('ओ', 55), ('म्', 55), ('प्', 55), ('य्', 53), ('ज्', 42), ('व्', 40), ('ऐ', 38)]


TOP 20 UNIGRAM SYLLABLES:  [('र', 88), ('अं', 70), ('क', 61), ('इ', 50), ('न', 46), ('स', 41), ('आ', 39), ('ल', 39), ('प', 36), ('ए', 34), ('ज', 31), ('ब', 30), ('के', 30), ('म', 29), ('य', 25), ('ओ', 24), ('त', 23), ('व', 23), ('है', 23), ('या', 22)]


TOP 20 UNIGRAM TOKENS: [('के', 26), ('प', 20), ('ब', 20), ('में', 2

## mBERT TOKENIZER FOR MAX_LENGTH - 2000 

In [25]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokens6 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(contents, max_length=2000, truncation=True, padding='max_length')))

In [26]:
print("mBERT TOKENIZER FOR VOCAB SIZE 2000: ")
processing(tokens6)

mBERT TOKENIZER FOR VOCAB SIZE 2000: 
GENETRATED TOKENS (TOP 50):  ['[CLS]', 'आ', '##वे', '##दन', 'करने', 'की', 'आ', '##ख', '##िर', '##ी', 'त', '##ारी', '##ख', '31', 'जनवरी', ',', '2020', 'है', '।', 'इ', '##तन', '##ी', 'दु', '##आ', 'कर', 'दो', 'हम', '##ारे', 'लिए', 'कि', 'ज', '##ित', '##ना', 'प', '##्या', '##र', 'दुनिया', 'ने', 'आ', '##प', '##को', 'दिया', 'है', ',', 'ब', '##स', 'उ', '##तन', '##ा', 'ही']


TOP 20 UNIGRAM CHARACTERS:  [('अ', 1277), ('आ', 349), ('क्', 292), ('ए', 285), ('र्', 248), ('इ', 208), ('ई', 168), ('न्', 166), ('स्', 156), ('अं', 141), ('ह्', 131), ('म्', 123), ('ल्', 119), ('त्', 109), ('य्', 109), ('ओ', 106), ('प्', 99), ('व्', 81), ('ज्', 79), ('द्', 72)]


TOP 20 UNIGRAM SYLLABLES:  [('र', 187), ('अं', 131), ('क', 105), ('इ', 98), ('स', 88), ('आ', 85), ('न', 83), ('प', 72), ('ए', 60), ('ल', 60), ('म', 59), ('ज', 54), ('ब', 52), ('व', 50), ('य', 49), ('उ', 47), ('ट', 47), ('ने', 45), ('है', 44), ('के', 44)]


TOP 20 UNIGRAM TOKENS: [('र', 38), ('के', 38), ('स',

## Indic-BERT TOKENIZER (MAX-LENGTH-1000):

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
tokens7 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sen, max_length=1000, truncation=True)))
# print(tokens7)

In [ ]:
print("Indic-BERT TOKENIZER FOR 1000: ")
processing(tokens7)

## Indic-BERT TOKENIZER (MAX-LENGTH-2000):

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
tokens9 = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sen, max_length=2000, truncation=True)))
# print(tokens7)

In [ ]:
print("Indic-BERT TOKENIZER FOR 2000: ")
processing(tokens9)

## WHITE SPACE TOKENIZER

In [27]:
tokens8 = contents.split()
print("WHITE SPACE TOKENIZER: ")
processing(tokens8)

WHITE SPACE TOKENIZER: 
GENETRATED TOKENS (TOP 50):  ['आवेदन', 'करने', 'की', 'आखिरी', 'तारीख', '31', 'जनवरी,', '2020', 'है।', 'इतनी', 'दुआ', 'कर', 'दो', 'हमारे', 'लिए', 'कि', 'जितना', 'प्यार', 'दुनिया', 'ने', 'आपको', 'दिया', 'है,', 'बस', 'उतना', 'ही', 'हमें', 'भी', 'मिल', 'जाए|”', 'मोदी', 'सरकार', 'के', 'पहले', 'कार्यकाल', 'में', 'भी', 'तीन', 'तलाक', 'को', 'लेकर', 'बिल', 'लाया', 'गया', 'था,', 'हालांकि', 'तब', 'यह', 'राज्यसभा', 'में']


TOP 20 UNIGRAM CHARACTERS:  [('अ', 7012291), ('आ', 2991109), ('ए', 2318442), ('क्', 2219964), ('र्', 2115649), ('ई', 1460305), ('इ', 1432973), ('न्', 1334448), ('स्', 1283708), ('अं', 1201207), ('ह्', 1133159), ('म्', 1053237), ('त्', 980066), ('ल्', 919917), ('ओ', 896588), ('प्', 805896), ('य्', 752819), ('व्', 624743), ('द्', 607633), ('उ', 587149)]


TOP 20 UNIGRAM SYLLABLES:  [('र', 1002121), ('अं', 989071), ('क', 584168), ('न', 493166), ('स', 483272), ('के', 404884), ('प', 382596), ('ने', 327692), ('ल', 323888), ('त', 307274), ('का', 304973), ('है',